In [65]:
import streamlit as st
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
df = pd.read_csv('/home/kar911/programming1/recommendation_system/cleaned.csv')
df=df.dropna(axis=0,how='any',subset=['clean_sorted'])

In [ ]:
['level', #one hot
  'subject',#one hot
 'Free/Paid'#one hot
 'rating',
 'price',
 'subject',#one hot
 'Free/Paid'#one hot
]

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
print(csr_matrix(b))

  (0, 2)	1
  (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 3)	1
  (1, 1)	1


In [66]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assume we have a Pandas DataFrame called data with multiple categorical columns to be one-hot encoded

# Create a OneHotEncoder object to encode the categorical data
encoder = OneHotEncoder(sparse_output=False,drop='first')

# Fit and transform the encoder on the data
data_encoded = encoder.fit_transform(df[['level','subject','Free/Paid']])

# The output data is a new DataFrame with the one-hot encoded features
data_encoded = pd.DataFrame(data_encoded)

data_encoded.head()

,0,1,2,3,4,5,6
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [29]:
df.head()

,Unnamed: 0,course_id,course_title,url,price,num_subscribers,num_reviews,num_lectures,level,rating,content_duration,published_timestamp,subject,Date,Free/Paid,clean_sorted,vid
0,0,41295,Learn HTML5 Programming From Scratch,https://www.udemy.com/learn-html5-programming-...,0,268923,8629,45,Beginner Level,0.82,10.5,2013-02-14T07:03:41Z,Web Development,2013-02-14,Free,Learn HTML Programming Scratch,vid1
1,1,59014,Coding for Entrepreneurs Basic,https://www.udemy.com/coding-for-entrepreneurs...,0,161029,279,27,Expert Level,0.69,3.5,2013-06-09T15:51:55Z,Web Development,2013-06-09,Free,Coding Entrepreneurs Basic,vid2
2,2,625204,The Web Developer Bootcamp,https://www.udemy.com/the-web-developer-bootcamp/,200,121584,27445,342,Beginner Level,0.89,43.0,2015-11-02T21:13:27Z,Web Development,2015-11-02,Paid,Web Developer Bootcamp,vid3
3,3,173548,Build Your First Website in 1 Week with HTML5 ...,https://www.udemy.com/build-your-first-website...,0,120291,5924,30,All Levels,0.78,3.0,2014-04-08T16:21:30Z,Web Development,2014-04-08,Free,Build First Website Week HTML CSS,vid4
4,4,764164,The Complete Web Developer Course 2.0,https://www.udemy.com/the-complete-web-develop...,200,114512,22412,304,Beginner Level,0.55,30.5,2016-03-08T22:28:36Z,Web Development,2016-03-08,Paid,Complete Web Developer Course,vid1


In [67]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

# Define the columns to normalize
cols_to_normalize = [ 'rating','price']

# Fit the scaler on the selected columns and transform the data
df[cols_to_normalize]= scaler.fit_transform(df[cols_to_normalize])


In [68]:
df[cols_to_normalize]

,rating,price
0,0.092308,-0.600000
1,-0.107692,-0.600000
2,0.200000,2.066667
3,0.030769,-0.600000
4,-0.323077,2.066667
...,...,...
3664,0.292308,-0.066667
3665,0.292308,0.400000
3666,-0.615385,0.600000
3667,-0.938462,0.066667


In [37]:
xx=df[cols_to_normalize].to_numpy()

In [38]:
xx

array([[ 0.09230769, -0.6       ],
       [-0.10769231, -0.6       ],
       [ 0.2       ,  2.06666667],
       ...,
       [-0.61538462,  0.6       ],
       [-0.93846154,  0.06666667],
       [-0.89230769,  0.06666667]])

In [40]:
cc=data_encoded.to_numpy()

In [41]:
cc.shape

(3648, 7)

In [42]:
vectorizer = CountVectorizer()
content_data = vectorizer.fit_transform(df['clean_sorted']).toarray()


In [43]:
all_data = np.concatenate((content_data, cc), axis=1)
all_data = np.concatenate((all_data, xx), axis=1)

In [56]:
cosine_sim = cosine_similarity(all_data)

In [57]:
cosine_sim1 = cosine_similarity(all_data,all_data)

In [46]:
all_data.shape

(3648, 3470)

In [48]:
all_data.shape

(3648, 3470)

In [50]:
import pickle
file_path = "cosine_similarity_model.pkl"

# Save the model using pickle
with open(file_path, 'wb') as f:
    pickle.dump(cosine_sim, f)
    
print("Model saved successfully!")

Model saved successfully!


In [61]:
def get_recommendations(title, cosine_sim=cosine_sim):
    indices = pd.Series(df.index, index=df['clean_sorted'])
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:6]
    movie_indices = [i[0] for i in sim_scores]
    return df[['course_title','url','price','num_subscribers','num_reviews','num_lectures','level','rating','subject','Free/Paid','vid','content_duration']].iloc[movie_indices]

In [52]:
cosine_sim.shape

(3648, 3648)

In [58]:
cosine_sim1.shape

(3648, 3648)

array([3648, 3648, 3648, ..., 3648, 3648, 3648])

In [63]:
def x(strr):
    text = strr.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = ' '.join([word for word in words if word.lower() not in stop_words])
    return re.sub(r'[^A-Za-z ]', '', filtered_words)

In [64]:
get_recommendations(x('Learn HTML5 Programming From Scratch'))

,course_title,url,price,num_subscribers,num_reviews,num_lectures,level,rating,subject,Free/Paid,vid,content_duration
0,Learn HTML5 Programming From Scratch,https://www.udemy.com/learn-html5-programming-...,-0.600000,268923,8629,45,Beginner Level,0.092308,Web Development,Free,vid1,10.5
211,Learn HTML And CSS From Scratch,https://www.udemy.com/html-and-css-from-scratch/,-0.600000,11533,240,18,Beginner Level,-0.846154,Web Development,Free,vid4,2.0
543,Learn HTML Basics From Scratch,https://www.udemy.com/learn-html-from-scratch/,-0.600000,5004,52,21,All Levels,0.353846,Web Development,Free,vid4,3.0
626,Learn JavaScript from scratch,https://www.udemy.com/learn-javascript-from-sc...,-0.333333,4193,92,44,Beginner Level,0.200000,Web Development,Paid,vid3,2.5
367,HTML Introduction Course: Learn HTML in 2 hour...,https://www.udemy.com/html-introduction-course...,-0.600000,7304,368,28,Beginner Level,-0.184615,Web Development,Free,vid4,2.0
554,HTML CSS Learn to create a website from scratch,https://www.udemy.com/html-css-learn-to-create...,0.066667,4892,49,32,Beginner Level,0.353846,Web Development,Paid,vid3,2.0
